In [ ]:
%cd yolov5 
!pip install -q -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17511, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 17511 (delta 5), reused 0 (delta 0), pack-reused 17493 (from 3)
Receiving objects: 100% (17511/17511), 16.65 MiB | 31.29 MiB/s, done.
Resolving deltas: 100% (11996/11996), done.
/kaggle/working/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.5 MB/s eta 0:00:00:00:0100:01


In [2]:
!pip install -q supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 4.8 MB/s eta 0:00:00a 0:00:01


# Define your paths here

In [3]:
weights_path = "/kaggle/input/yolov5m/pytorch/default/1/best.pt"
images_path = "/kaggle/input/test-detection-and-tracking/Test_detection_and_tracking/RGB/images"
videos_path = "/kaggle/input/test-detection-and-tracking/Test_detection_and_tracking/RGB/videos"
project_path = "/kaggle/working"                                # Project root directory
device = "0,1"                                                  # The device on which to run detection task
submission_filename = "submission_detection_tracking_RGB.csv"

devices = ["0,1", "cpu"]                                        # Tracking task will run on gpu and cpu both

In [4]:
!echo "$weights_path"
!echo "$images_path"
!echo "$videos_path"

/kaggle/input/yolov5m/pytorch/default/1/best.pt
/kaggle/input/test-detection-and-tracking/Test_detection_and_tracking/RGB/images
/kaggle/input/test-detection-and-tracking/Test_detection_and_tracking/RGB/videos


In [5]:
detect = r"""
# Ultralytics 🚀 AGPL-3.0 License - https://ultralytics.com/license

import argparse
import csv
import os
import platform
import sys
from pathlib import Path

import torch

FILE = Path(__file__).resolve()
ROOT = FILE.parents[0]  # YOLOv5 root directory
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

from ultralytics.utils.plotting import Annotator, colors, save_one_box

from models.common import DetectMultiBackend
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadScreenshots, LoadStreams
from utils.general import (
    LOGGER,
    Profile,
    check_file,
    check_img_size,
    check_imshow,
    check_requirements,
    colorstr,
    cv2,
    increment_path,
    non_max_suppression,
    print_args,
    scale_boxes,
    strip_optimizer,
    xyxy2xywh,
)
from utils.torch_utils import select_device, smart_inference_mode


def box_iou2(boxes1, boxes2):
    area1 = (boxes1[:, 2] - boxes1[:, 0]) * (boxes1[:, 3] - boxes1[:, 1])
    area2 = (boxes2[:, 2] - boxes2[:, 0]) * (boxes2[:, 3] - boxes2[:, 1])

    lt = torch.max(boxes1[:, None, :2], boxes2[:, :2])  # (N, M, 2)
    rb = torch.min(boxes1[:, None, 2:], boxes2[:, 2:])  # (N, M, 2)

    wh = (rb - lt).clamp(min=0)  # (N, M, 2)
    inter = wh[:, :, 0] * wh[:, :, 1]  # (N, M)

    union = area1[:, None] + area2 - inter

    iou = inter / (union + 1e-6)
    return iou

def remove_enclosing_boxes_vectorized(preds, area_thresh=1.05):
    if preds.shape[0] <= 1:
        return preds

    boxes = preds[:, :4]  # (N, 4)
    confs = preds[:, 4]   # (N,)
    labels = preds[:, 5]  # (N,)
    N = preds.shape[0]

    x1 = boxes[:, 0].unsqueeze(1)
    y1 = boxes[:, 1].unsqueeze(1)
    x2 = boxes[:, 2].unsqueeze(1)
    y2 = boxes[:, 3].unsqueeze(1)

    area = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])  # (N,)

    # Broadcast comparisons
    encloses = (
        (x1 <= x1.T) & (y1 <= y1.T) &
        (x2 >= x2.T) & (y2 >= y2.T)
    )  # (N, N)

    larger_area = area.unsqueeze(1) > area.unsqueeze(0) * area_thresh
    same_class = labels.unsqueeze(1) == labels.unsqueeze(0)
    higher_conf = confs.unsqueeze(1) < confs.unsqueeze(0)

    # i encloses j AND i is larger AND same class AND j has higher conf → suppress i
    suppress = encloses & larger_area & same_class & higher_conf

    to_keep = ~(suppress.any(dim=1))  # Keep boxes not suppressed by any better one

    return preds[to_keep]


def temporal_completion(filtered_all, iou_thresh=0.4, window=3):
    for i in range(len(filtered_all)):
        current = filtered_all[i]
        if current.shape[0] > 0:
            continue  # already has detections

        to_add = []
        for offset in range(1, window + 1):
            prev_idx = i - offset
            next_idx = i + offset
            if prev_idx < 0 or next_idx >= len(filtered_all):
                continue

            prev = filtered_all[prev_idx]
            nxt = filtered_all[next_idx]
            if prev.shape[0] == 0 or nxt.shape[0] == 0:
                continue

            ious = box_iou2(prev[:, :4], nxt[:, :4])
            class_match = prev[:, None, 5] == nxt[:, None, 5].T
            ious[~class_match] = 0.0
            matched = (ious >= iou_thresh)

            for p_idx, n_idx in torch.nonzero(matched, as_tuple=False):
                box1 = prev[p_idx]
                box2 = nxt[n_idx]
                interpolated_box = (box1[:4] + box2[:4]) / 2
                interpolated_conf = min(box1[4], box2[4]) * 0.9  # slightly lower confidence
                interpolated_label = box1[5]
                new_det = torch.cat([interpolated_box, interpolated_conf.unsqueeze(0), interpolated_label.unsqueeze(0)])
                to_add.append(new_det.unsqueeze(0))

        if to_add:
            filled = torch.cat(to_add, dim=0)
            filled = remove_enclosing_boxes_vectorized(filled)
            filtered_all[i] = filled
    
    return filtered_all


max_label_dist = 0
gt_included_count = 0
def temporal_filter_preds(all_preds, iou_thresh=0.4, window=3, boost_conf=0.1, min_matches=1):
    filtered_all = []
    max_label_iou = 0

    def convert_labels_to_xyxy(labels):
        cx, cy, w, h = labels[:, 1], labels[:, 2], labels[:, 3], labels[:, 4]
        x1, y1 = cx - w / 2, cy - h / 2
        x2, y2 = cx + w / 2, cy + h / 2
        return torch.stack([x1, y1, x2, y2], dim=1)

    def get_pred_boxes(preds):
        return preds[:, :4]

    for i, preds in enumerate(all_preds):
        if preds.shape[0] == 0:
            filtered_all.append(preds)
            continue

        pred_boxes = get_pred_boxes(preds)  # (N, 4)
        match_counts = torch.zeros(preds.shape[0], device=preds.device)

        for offset in range(-window, window + 1):
            if offset == 0:
                continue
            j = i + offset
            if 0 <= j < len(all_preds):
                neighbor_preds = all_preds[j]
                if neighbor_preds.shape[0] == 0:
                    continue
                neighbor_boxes = get_pred_boxes(neighbor_preds)

                ious = box_iou2(pred_boxes, neighbor_boxes)  # (N, M)
                class_match = preds[:, None, 5] == neighbor_preds[None, :, 5]
                ious[~class_match] = 0.0  # force non-class matches to zero IoU
                match_counts += (ious >= iou_thresh).any(dim=1).float()

        keep_mask = match_counts >= min_matches
        kept = preds[keep_mask]

        boost_mask = match_counts[keep_mask] >= window * 2
        if boost_mask.any():
            kept[boost_mask, 4] = torch.clamp(kept[boost_mask, 4] + boost_conf, 0.0, 1.0)

        kept = remove_enclosing_boxes_vectorized(kept)
        filtered_all.append(kept)
    return filtered_all


@smart_inference_mode()
def run(
    weights=ROOT / "yolov5s.pt",  # model path or triton URL
    source=ROOT / "data/images",  # file/dir/URL/glob/screen/0(webcam)
    data=ROOT / "data/coco128.yaml",  # dataset.yaml path
    imgsz=(640, 640),  # inference size (height, width)
    conf_thres=0.25,  # confidence threshold
    iou_thres=0.45,  # NMS IOU threshold
    max_det=1000,  # maximum detections per image
    device="",  # cuda device, i.e. 0 or 0,1,2,3 or cpu
    view_img=False,  # show results
    save_txt=False,  # save results to *.txt
    save_format=0,  # save boxes coordinates in YOLO format or Pascal-VOC format (0 for YOLO and 1 for Pascal-VOC)
    save_csv=False,  # save results in CSV format
    save_conf=False,  # save confidences in --save-txt labels
    save_crop=False,  # save cropped prediction boxes
    nosave=False,  # do not save images/videos
    classes=None,  # filter by class: --class 0, or --class 0 2 3
    agnostic_nms=False,  # class-agnostic NMS
    augment=False,  # augmented inference
    visualize=False,  # visualize features
    update=False,  # update all models
    project=ROOT / "runs/detect",  # save results to project/name
    name="exp",  # save results to project/name
    exist_ok=False,  # existing project/name ok, do not increment
    line_thickness=3,  # bounding box thickness (pixels)
    hide_labels=False,  # hide labels
    hide_conf=False,  # hide confidences
    half=False,  # use FP16 half-precision inference
    dnn=False,  # use OpenCV DNN for ONNX inference
    vid_stride=1,  # video frame-rate stride
):
    
    source = str(source)
    save_img = not nosave and not source.endswith(".txt")  # save inference images
    is_file = Path(source).suffix[1:] in (IMG_FORMATS + VID_FORMATS)
    is_url = source.lower().startswith(("rtsp://", "rtmp://", "http://", "https://"))
    webcam = source.isnumeric() or source.endswith(".streams") or (is_url and not is_file)
    screenshot = source.lower().startswith("screen")
    if is_url and is_file:
        source = check_file(source)  # download

    # Directories
    save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # increment run
    (save_dir / "labels" if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Load model
    device = select_device(device)
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
    stride, names, pt = model.stride, model.names, model.pt
    imgsz = check_img_size(imgsz, s=stride)  # check image size

    # Dataloader
    bs = 1  # batch_size
    if webcam:
        view_img = check_imshow(warn=True)
        dataset = LoadStreams(source, img_size=imgsz, stride=stride, auto=pt, vid_stride=vid_stride)
        bs = len(dataset)
    elif screenshot:
        dataset = LoadScreenshots(source, img_size=imgsz, stride=stride, auto=pt)
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt, vid_stride=vid_stride)
    vid_path, vid_writer = [None] * bs, [None] * bs

    # Run inference
    model.warmup(imgsz=(1 if pt or model.triton else bs, 3, *imgsz))  # warmup
    seen, windows, dt = 0, [], (Profile(device=device), Profile(device=device), Profile(device=device))
    for p in dt:
        p.dt = 0.0
    for path, im, im0s, vid_cap, s in dataset:
        # print(im0s.shape)
        for p in dt:
            p.dt = 0.0
        with dt[0]:
            im = torch.from_numpy(im).to(model.device)
            im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
            im /= 255  # 0 - 255 to 0.0 - 1.0
            if len(im.shape) == 3:
                im = im[None]  # expand for batch dim
            if model.xml and im.shape[0] > 1:
                ims = torch.chunk(im, im.shape[0], 0)

        # Inference
        with dt[1]:
            visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
            if model.xml and im.shape[0] > 1:
                pred = None
                for image in ims:
                    if pred is None:
                        pred = model(image, augment=augment, visualize=visualize).unsqueeze(0)
                    else:
                        pred = torch.cat((pred, model(image, augment=augment, visualize=visualize).unsqueeze(0)), dim=0)
                pred = [pred, None]
            else:
                pred = model(im, augment=augment, visualize=visualize)
        # NMS
        with dt[2]:
            pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)

        # pred = temporal_filter_preds(pred)
        # pred = temporal_completion(pred)
        # Second-stage classifier (optional)
        # pred = utils.general.apply_classifier(pred, classifier_model, im, im0s)

        # Define the path for the CSV file
        csv_path = save_dir / "predictions.csv"

        # Create or append to the CSV file
        def write_to_csv(image_name, prediction, confidence):
            data = {"Image Name": image_name, "Prediction": prediction, "Confidence": confidence}
            file_exists = os.path.isfile(csv_path)
            with open(csv_path, mode="a", newline="") as f:
                writer = csv.DictWriter(f, fieldnames=data.keys())
                if not file_exists:
                    writer.writeheader()
                writer.writerow(data)

        # Process predictions
        for i, det in enumerate(pred):  # per image
            seen += 1
            if webcam:  # batch_size >= 1
                p, im0, frame = path[i], im0s[i].copy(), dataset.count
                s += f"{i}: "
            else:
                p, im0, frame = path, im0s.copy(), getattr(dataset, "frame", 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # im.jpg
            txt_path = str(save_dir / "labels" / p.stem) + ("" if dataset.mode == "image" else f"_{frame}")  # im.txt
            s += "{:g}x{:g} ".format(*im.shape[2:])  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            # print("gain: " + str(gn))
            imc = im0.copy() if save_crop else im0  # for save_crop
            annotator = Annotator(im0, line_width=line_thickness, example=str(names))
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_boxes(im.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, 5].unique():
                    n = (det[:, 5] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    c = int(cls)  # integer class
                    label = names[c] if hide_conf else f"{names[c]}"
                    confidence = float(conf)
                    confidence_str = f"{confidence:.2f}"
                    total_time_ms = sum(p.dt for p in dt) * 1000

                    if save_csv:
                        write_to_csv(p.name, label, confidence_str)

                    if save_txt:  # Write to file
                        if save_format == 0:
                            coords = (
                                (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()
                            )  # normalized xywh
                        else:
                            coords = (torch.tensor(xyxy).view(1, 4) / gn).view(-1).tolist()  # xyxy
                        # print(coords)
                        line = (cls, *coords, conf, total_time_ms) if save_conf else (cls, *coords, total_time_ms)  # label format
                        with open(f"{txt_path}.txt", "a") as f:
                            f.write(("%g " * len(line)).rstrip() % line + "\n")

                    if save_img or save_crop or view_img:  # Add bbox to image
                        c = int(cls)  # integer class
                        label = None if hide_labels else (names[c] if hide_conf else f"{names[c]} {conf:.2f}")
                        annotator.box_label(xyxy, label, color=colors(c, True))
                    if save_crop:
                        save_one_box(xyxy, imc, file=save_dir / "crops" / names[c] / f"{p.stem}.jpg", BGR=True)

            # Stream results
            im0 = annotator.result()
            if view_img:
                if platform.system() == "Linux" and p not in windows:
                    windows.append(p)
                    cv2.namedWindow(str(p), cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)  # allow window resize (Linux)
                    cv2.resizeWindow(str(p), im0.shape[1], im0.shape[0])
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_img:
                if dataset.mode == "image":
                    cv2.imwrite(save_path, im0)
                else:  # 'video' or 'stream'
                    if vid_path[i] != save_path:  # new video
                        vid_path[i] = save_path
                        if isinstance(vid_writer[i], cv2.VideoWriter):
                            vid_writer[i].release()  # release previous video writer
                        if vid_cap:  # video
                            fps = vid_cap.get(cv2.CAP_PROP_FPS)
                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        else:  # stream
                            fps, w, h = 30, im0.shape[1], im0.shape[0]
                        save_path = str(Path(save_path).with_suffix(".mp4"))  # force *.mp4 suffix on results videos
                        vid_writer[i] = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))
                    vid_writer[i].write(im0)

        # Print time (inference-only)
        # LOGGER.info(f"{s}{'' if len(det) else '(no detections), '}{dt[1].dt * 1e3:.1f}ms")

    # Print results
    #t = tuple(x.t / seen * 1e3 for x in dt)  # speeds per image
    #LOGGER.info(f"Speed: %.1fms pre-process, %.1fms inference, %.1fms NMS per image at shape {(1, 3, *imgsz)}" % t)
    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ""
        LOGGER.info(f"Results saved to {colorstr('bold', save_dir)}{s}")
    if update:
        strip_optimizer(weights[0])  # update model (to fix SourceChangeWarning)


def parse_opt():
    
    parser = argparse.ArgumentParser()
    parser.add_argument("--weights", nargs="+", type=str, default=ROOT / "yolov5s.pt", help="model path or triton URL")
    parser.add_argument("--source", type=str, default=ROOT / "data/images", help="file/dir/URL/glob/screen/0(webcam)")
    parser.add_argument("--data", type=str, default=ROOT / "data/coco128.yaml", help="(optional) dataset.yaml path")
    parser.add_argument("--imgsz", "--img", "--img-size", nargs="+", type=int, default=[640], help="inference size h,w")
    parser.add_argument("--conf-thres", type=float, default=0.25, help="confidence threshold")
    parser.add_argument("--iou-thres", type=float, default=0.45, help="NMS IoU threshold")
    parser.add_argument("--max-det", type=int, default=1000, help="maximum detections per image")
    parser.add_argument("--device", default="", help="cuda device, i.e. 0 or 0,1,2,3 or cpu")
    parser.add_argument("--view-img", action="store_true", help="show results")
    parser.add_argument("--save-txt", action="store_true", help="save results to *.txt")
    parser.add_argument(
        "--save-format",
        type=int,
        default=0,
        help="whether to save boxes coordinates in YOLO format or Pascal-VOC format when save-txt is True, 0 for YOLO and 1 for Pascal-VOC",
    )
    parser.add_argument("--save-csv", action="store_true", help="save results in CSV format")
    parser.add_argument("--save-conf", action="store_true", help="save confidences in --save-txt labels")
    parser.add_argument("--save-crop", action="store_true", help="save cropped prediction boxes")
    parser.add_argument("--nosave", action="store_true", help="do not save images/videos")
    parser.add_argument("--classes", nargs="+", type=int, help="filter by class: --classes 0, or --classes 0 2 3")
    parser.add_argument("--agnostic-nms", action="store_true", help="class-agnostic NMS")
    parser.add_argument("--augment", action="store_true", help="augmented inference")
    parser.add_argument("--visualize", action="store_true", help="visualize features")
    parser.add_argument("--update", action="store_true", help="update all models")
    parser.add_argument("--project", default=ROOT / "runs/detect", help="save results to project/name")
    parser.add_argument("--name", default="exp", help="save results to project/name")
    parser.add_argument("--exist-ok", action="store_true", help="existing project/name ok, do not increment")
    parser.add_argument("--line-thickness", default=3, type=int, help="bounding box thickness (pixels)")
    parser.add_argument("--hide-labels", default=False, action="store_true", help="hide labels")
    parser.add_argument("--hide-conf", default=False, action="store_true", help="hide confidences")
    parser.add_argument("--half", action="store_true", help="use FP16 half-precision inference")
    parser.add_argument("--dnn", action="store_true", help="use OpenCV DNN for ONNX inference")
    parser.add_argument("--vid-stride", type=int, default=1, help="video frame-rate stride")
    opt = parser.parse_args()
    opt.imgsz *= 2 if len(opt.imgsz) == 1 else 1  # expand
    print_args(vars(opt))
    return opt


def main(opt):
    
    check_requirements(ROOT / "requirements.txt", exclude=("tensorboard", "thop"))
    run(**vars(opt))


if __name__ == "__main__":
    opt = parse_opt()
    main(opt)
"""

with open("detect2.py", 'w') as f:
    f.write(detect)

In [6]:
!python detect2.py --weights "$weights_path" --source "$images_path" --conf-thres 0.2 --save-format 1 --device "$device" --save-txt --save-csv --save-conf --nosave

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
detect2: weights=['/kaggle/input/yolov5m/pytorch/default/1/best.pt'], source=/kaggle/input/test-detection-and-tracking/Test_detection_and_tracking/RGB/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.2, iou_thres=0.45, max_det=1000, device=0,1, view_img=False, save_txt=True, save_format=1, save_csv=True, save_conf=True, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-422-g2540fd4c Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                 

In [7]:
obj = r"""
import os
import sys
import cv2
import time
import torch
import argparse
import numpy as np
from pathlib import Path
from collections import Counter
import torch.backends.cudnn as cudnn
from utils.general import set_logging
from models.common import DetectMultiBackend
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, Profile, check_file, check_img_size, 
                            check_imshow, check_requirements, colorstr, cv2,
                           increment_path, non_max_suppression, print_args,
                            scale_boxes, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, time_sync

FILE = Path(__file__).resolve()
ROOT = FILE.parents[0] 
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT)) 
ROOT = Path(os.path.relpath(ROOT, Path.cwd())) 

#---------------Object Tracking---------------
import skimage
import supervision as sv

#-----------Object Blurring-------------------
blurratio = 40


#.................. Tracker Functions .................
'''Computer Color for every box and track'''
palette = (2 ** 11 - 1, 2 ** 15 - 1, 2 ** 20 - 1)
def compute_color_for_labels(label):
    color = [int(int(p * (label ** 2 - label + 1)) % 255) for p in palette]
    return tuple(color)


def bbox_rel(*xyxy):
    bbox_left = min([xyxy[0].item(), xyxy[2].item()])
    bbox_top = min([xyxy[1].item(), xyxy[3].item()])
    bbox_w = abs(xyxy[0].item() - xyxy[2].item())
    bbox_h = abs(xyxy[1].item() - xyxy[3].item())
    x_c = (bbox_left + bbox_w / 2)
    y_c = (bbox_top + bbox_h / 2)
    w = bbox_w
    h = bbox_h
    return x_c, y_c, w, h


def draw_boxes(img, bbox, identities=None, categories=None, 
                names=None, color_box=None,offset=(0, 0)):
    for i, box in enumerate(bbox):
        x1, y1, x2, y2 = [int(i) for i in box]
        x1 += offset[0]
        x2 += offset[0]
        y1 += offset[1]
        y2 += offset[1]
        cat = int(categories[i]) if categories is not None else 0
        id = int(identities[i]) if identities is not None else 0
        data = (int((box[0]+box[2])/2),(int((box[1]+box[3])/2)))
        label = str(id)

        if color_box:
            color = compute_color_for_labels(id)
            (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
            cv2.rectangle(img, (x1, y1), (x2, y2),color, 2)
            cv2.rectangle(img, (x1, y1 - 20), (x1 + w, y1), (255,191,0), -1)
            cv2.putText(img, label, (x1, y1 - 5),cv2.FONT_HERSHEY_SIMPLEX, 0.6, 
            [255, 255, 255], 1)
            cv2.circle(img, data, 3, color,-1)
        else:
            (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
            cv2.rectangle(img, (x1, y1), (x2, y2),(255,191,0), 2)
            cv2.rectangle(img, (x1, y1 - 20), (x1 + w, y1), (255,191,0), -1)
            cv2.putText(img, label, (x1, y1 - 5),cv2.FONT_HERSHEY_SIMPLEX, 0.6, 
            [255, 255, 255], 1)
            cv2.circle(img, data, 3, (255,191,0),-1)
    return img
#..............................................................................


@torch.no_grad()
def detect(weights=ROOT / 'yolov5n.pt',
        source=ROOT / 'yolov5/data/images', 
        data=ROOT / 'yolov5/data/coco128.yaml',  
        imgsz=(640, 640),conf_thres=0.25,iou_thres=0.45,  
        max_det=1000, device='cpu',  view_img=False,  
        save_txt=False, save_conf=False, save_crop=False, 
        nosave=False, classes=None,  agnostic_nms=False,  
        augment=False, visualize=False,  update=False,  
        project=ROOT / 'runs/detect',  name='exp',  
        exist_ok=False, line_thickness=2,hide_labels=False,  
        hide_conf=False,half=False,dnn=False,display_labels=False,
        blur_obj=False,color_box = False,):
    
    save_img = not nosave and not source.endswith('.txt') 
    
    #.... Initialize SORT .... 
    
    track_color_id = 0

    byte_tracker = sv.ByteTrack()
    smoother = sv.DetectionsSmoother()
    label = sv.LabelAnnotator()
    trace_annotator = sv.TraceAnnotator()
    video_info = sv.VideoInfo.from_video_path(source)
    frame_generator = sv.get_video_frames_generator(source)
    fps_monitor = sv.FPSMonitor()
    #......................... 
    
    
    webcam = source.isnumeric() or source.endswith('.txt') or source.lower().startswith(
        ('rtsp://', 'rtmp://', 'http://', 'https://'))

    save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  

    set_logging()
    device = select_device(device)
    half &= device.type != 'cpu'  

    device = select_device(device)
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data)
    stride, names, pt, jit, onnx, engine = model.stride, model.names, model.pt, model.jit, model.onnx, model.engine
    imgsz = check_img_size(imgsz, s=stride)  

    half &= (pt or jit or onnx or engine) and device.type != 'cpu'  
    if pt or jit:
        model.model.half() if half else model.model.float()

    if webcam:
        cudnn.benchmark = True  
        dataset = LoadStreams(source, img_size=imgsz, stride=stride, auto=pt)
        bs = len(dataset) 
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt)
        bs = 1 
    vid_path, vid_writer = [None] * bs, [None] * bs
    
    t0 = time.time()
    
    dt, seen = [0.0, 0.0, 0.0], 0
    
    with sv.VideoSink(target_path="output.mp4", video_info=video_info) as sink:
        for path, im, im0s, vid_cap, s in dataset:

            svframe = next(frame_generator)
            t1 = time_sync()
            im = torch.from_numpy(im).to(device)
            im = im.half() if half else im.float()  # uint8 to fp16/32
            im /= 255  # 0 - 255 to 0.0 - 1.0
            if len(im.shape) == 3:
                im = im[None]  # expand for batch dim
            t2 = time_sync()
            dt[0] += t2 - t1
    
            # Inference
            visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
            pred = model(im, augment=augment, visualize=visualize)
            t3 = time_sync()
            dt[1] += t3 - t2
    
            # NMS
            pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
            dt[2] += time_sync() - t3
    
            
            for i, det in enumerate(pred):  # per image
                seen += 1
                if webcam:  # batch_size >= 1
                    p, im0, frame = path[i], im0s[i].copy(), dataset.count
                    s += f'{i}: '
                else:
                    p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)
    
                p = Path(p)
                save_path = str(save_dir / p.name)
                txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # im.txt
                s += '%gx%g ' % im.shape[2:]
                gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]
                # print(gn)
                imc = im0.copy() if save_crop else im0
                annotator = Annotator(im0, line_width=line_thickness, example=str(names))
                if len(det):
                    det[:, :4] = scale_boxes(im.shape[2:], det[:, :4], im0.shape).round()
                    for c in det[:, -1].unique():
                        n = (det[:, -1] == c).sum()
                        s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "
    
                    # Write results
                    for *xyxy, conf, cls in reversed(det):
                        if blur_obj:
                            crop_obj = im0[int(xyxy[1]):int(xyxy[3]),int(xyxy[0]):int(xyxy[2])]
                            blur = cv2.blur(crop_obj,(blurratio,blurratio))
                            im0[int(xyxy[1]):int(xyxy[3]),int(xyxy[0]):int(xyxy[2])] = blur
                        else:
                            continue
                    #..................USE TRACK FUNCTION....................
                    
                    
                    # Run bytetrack
                    pred_np = det.cpu().numpy()
                    xyxy = pred_np[:, :4]
                    confidence = pred_np[:, 4]
                    class_id = pred_np[:, 5].astype(int)
                    detections = sv.Detections(
                        xyxy=xyxy,
                        confidence=confidence,
                        class_id=class_id
                    )
                    detections = byte_tracker.update_with_detections(detections)
                    detections = smoother.update_with_detections(detections)
                    # print("Track_id: " + str(detections.tracker_id) + ", track_confidence: " + str(detections.confidence))
                    labels = [ f"{tracker_id}, {confidence:.2f}" for tracker_id, confidence in zip(detections.tracker_id, detections.confidence) ]
                    annotated_frame = label.annotate(scene=svframe.copy(), detections=detections, labels=labels)
                    annotated_frame = trace_annotator.annotate(scene=annotated_frame, detections=detections)
                    sink.write_frame(frame=annotated_frame)
                    
                        
                    if save_txt:
                        os.makedirs(save_dir / "labels", exist_ok=True)
                        frame_txt_path = f"{save_dir / 'labels' / p.stem}_{frame}.txt"
                        with open(frame_txt_path, 'w') as f:
                            for box, tracker_id, class_id, track_conf, det_conf in zip(
                                    detections.xyxy, detections.tracker_id, detections.class_id, detections.confidence, confidence):
                                coords = (torch.tensor(box).view(1, 4) / gn).view(-1).tolist()  # xyxy
                                x1, y1, x2, y2 = coords
                                line = f"{int(tracker_id)} {int(class_id)} {track_conf:.2f} {det_conf:.2f} {x1:.4f} {y1:.4f} {x2:.4f} {y2:.4f}\n"
                                f.write(line)


    
                if view_img:
                    cv2.imshow(str(p), im0)
                    cv2.waitKey(1) 
                if save_img:
                    if dataset.mode == 'image':
                        cv2.imwrite(save_path, im0)
                    else:
                        if vid_path != save_path: 
                            vid_path = save_path
                            if isinstance(vid_writer, cv2.VideoWriter):
                                vid_writer.release()  
                            if vid_cap: 
                                fps = vid_cap.get(cv2.CAP_PROP_FPS)
                                w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                                h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                            else:
                                fps, w, h = 30, im0.shape[1], im0.shape[0]
                                save_path += '.mp4'
                            vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                        vid_writer.write(im0)
            #print("Frame Processing!")
            fps_monitor.tick()
    print("Video Exported Success")
    print(str(fps_monitor.fps))

    if update:
        strip_optimizer(weights)
    
    if vid_cap:
        vid_cap.release()


def parse_opt():
    parser = argparse.ArgumentParser()
    parser.add_argument('--weights', nargs='+', type=str, default=ROOT / 'yolov5s.pt', help='model path(s)')
    parser.add_argument('--source', type=str, default=ROOT / 'data/images', help='file/dir/URL/glob, 0 for webcam')
    parser.add_argument('--data', type=str, default=ROOT / 'data/coco128.yaml', help='(optional) dataset.yaml path')
    parser.add_argument('--imgsz', '--img', '--img-size', nargs='+', type=int, default=[640], help='inference size h,w')
    parser.add_argument('--conf-thres', type=float, default=0.25, help='confidence threshold')
    parser.add_argument('--iou-thres', type=float, default=0.45, help='NMS IoU threshold')
    parser.add_argument('--max-det', type=int, default=1000, help='maximum detections per image')
    parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
    parser.add_argument('--view-img', action='store_true', help='show results')
    parser.add_argument('--save-txt', action='store_true', help='save results to *.txt')
    parser.add_argument('--save-conf', action='store_true', help='save confidences in --save-txt labels')
    parser.add_argument('--save-crop', action='store_true', help='save cropped prediction boxes')
    parser.add_argument('--nosave', action='store_true', help='do not save images/videos')
    parser.add_argument('--classes', nargs='+', type=int, help='filter by class: --classes 0, or --classes 0 2 3')
    parser.add_argument('--agnostic-nms', action='store_true', help='class-agnostic NMS')
    parser.add_argument('--augment', action='store_true', help='augmented inference')
    parser.add_argument('--visualize', action='store_true', help='visualize features')
    parser.add_argument('--update', action='store_true', help='update all models')
    parser.add_argument('--project', default=ROOT / 'runs/detect', help='save results to project/name')
    parser.add_argument('--name', default='exp', help='save results to project/name')
    parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
    parser.add_argument('--line-thickness', default=3, type=int, help='bounding box thickness (pixels)')
    parser.add_argument('--hide-labels', default=False, action='store_true', help='hide labels')
    parser.add_argument('--hide-conf', default=False, action='store_true', help='hide confidences')
    parser.add_argument('--half', action='store_true', help='use FP16 half-precision inference')
    parser.add_argument('--dnn', action='store_true', help='use OpenCV DNN for ONNX inference')
    parser.add_argument('--blur-obj', action='store_true', help='Blur Detected Objects')
    parser.add_argument('--color-box', action='store_true', help='Change color of every box and track')
    opt = parser.parse_args()
    opt.imgsz *= 2 if len(opt.imgsz) == 1 else 1  # expand
    print_args(vars(opt))
    return opt


def main(opt):
    check_requirements(exclude=('tensorboard', 'thop'))
    detect(**vars(opt))


if __name__ == "__main__":
    opt = parse_opt()
    main(opt)
"""

with open('obj_det_and_trk2.py', 'w') as f:
    f.write(obj)

In [8]:
import os
import subprocess
import glob

conf_thres = 0.01

fps_results = {}

videos = glob.glob(os.path.join(videos_path, "*.mp4"))

for video in videos:
    video_name = os.path.splitext(os.path.basename(video))[0]
    fps_results[video_name] = {}
    print("Processing video: " + video_name)

    for device in devices:
        result = subprocess.run(
            ["python", "obj_det_and_trk2.py", "--weights", weights_path, "--source",  video, "--imgsz", "320", "--conf-thres", str(conf_thres), "--save-txt", "--save-conf", "--project", project_path, "--device", device],
            capture_output=True,
            text=True
        )

        try:
            fps_value = float(result.stdout.strip().split('\n')[-1])
        except Exception as e:
            print(f"Error extracting FPS for {video} on {device}: {e}")
            fps_value = -1

        fps_results[video_name][device] = fps_value

# Optional: Print or save the results
print(fps_results)

Processing video: V_DRONE_099
Processing video: V_BIRD_043
Processing video: V_BIRD_050
Processing video: V_BIRD_051
Processing video: V_BIRD_045
Processing video: V_DRONE_111
Processing video: V_DRONE_104
Processing video: V_DRONE_101
Processing video: V_BIRD_047
Processing video: V_DRONE_114
Processing video: V_DRONE_105
Processing video: V_DRONE_108
{'V_DRONE_099': {'0,1': 63.334480201126695, 'cpu': 10.443430662878828}, 'V_BIRD_043': {'0,1': 66.6382666221393, 'cpu': 11.100440188067248}, 'V_BIRD_050': {'0,1': 64.50725269557364, 'cpu': 10.024978349672631}, 'V_BIRD_051': {'0,1': 70.56630212515533, 'cpu': 11.744176697646747}, 'V_BIRD_045': {'0,1': 68.40427089177041, 'cpu': 8.444900102603455}, 'V_DRONE_111': {'0,1': 71.14050995320736, 'cpu': 11.485375813396452}, 'V_DRONE_104': {'0,1': 59.58089440431222, 'cpu': 9.54690013533739}, 'V_DRONE_101': {'0,1': 63.17631029877135, 'cpu': 10.854303300071493}, 'V_BIRD_047': {'0,1': 62.53915133281106, 'cpu': 9.130903722004303}, 'V_DRONE_114': {'0,1': 

In [ ]:
# !python obj_det_and_trk2.py --weights /kaggle/input/yolov5m/pytorch/default/1/best.pt --source /kaggle/input/test-detection-and-tracking/Test_detection_and_tracking/RGB/videos/V_DRONE_101.mp4 --imgsz 320 --conf-thres 0.01 --save-txt --save-conf --project=/kaggle/working

In [ ]:
import os
import glob
import numpy as np
from collections import defaultdict

base_dir = "/kaggle/working"
exp_dirs = sorted(glob.glob(os.path.join(base_dir, "exp*")), key=os.path.getmtime)

print(f"Found {len(exp_dirs)} exp directories.")

for exp_dir in exp_dirs:
    labels_dir = os.path.join(exp_dir, "labels")
    if not os.path.isdir(labels_dir):
        print(f"Skipping {exp_dir}, no 'labels' folder found.")
        continue

    print(f"Processing label directory: {labels_dir}")

    # Collect bounding box area per track_id per frame
    track_areas = defaultdict(list)  # track_id: list of (frame_number, area)
    file_list = sorted(glob.glob(os.path.join(labels_dir, "*.txt")))
    frame_index_map = {file: idx for idx, file in enumerate(file_list)}

    for txt_file in file_list:
        frame_id = frame_index_map[txt_file]
        with open(txt_file, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 8:
                    continue
                track_id = int(parts[0])
                x1, y1, x2, y2 = map(float, parts[4:8])
                area = (x2 - x1) * (y2 - y1) * 320 * 256
                track_areas[track_id].append((frame_id, area))

    # Compute trend per track_id
    track_trend = {}
    for track_id, entries in track_areas.items():
        entries.sort()
        frame_ids, areas = zip(*entries)
        if len(areas) < 3:
            trend = "stationary"
        else:
            slope = np.polyfit(frame_ids, areas, 1)[0]
            if slope > 0.0:
                trend = "approaching"
            else:
                trend = "receding"
        track_trend[track_id] = trend

    # Rewrite each txt file with trend info as final column
    for txt_file in file_list:
        new_lines = []
        with open(txt_file, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 8:
                    new_lines.append(line)
                    continue
                track_id = int(parts[0])
                trend = track_trend.get(track_id, "stationary")
                new_line = line.strip() + f" {trend}\n"
                new_lines.append(new_line)
        with open(txt_file, "w") as f:
            f.writelines(new_lines)

    print(f"Updated all label files in {labels_dir} with trend column.")

print("All exp directories processed.")

Found 24 exp directories.
Processing label directory: /kaggle/working/exp/labels
Updated all label files in /kaggle/working/exp/labels with trend column.
Processing label directory: /kaggle/working/exp2/labels
Updated all label files in /kaggle/working/exp2/labels with trend column.
Processing label directory: /kaggle/working/exp3/labels
Updated all label files in /kaggle/working/exp3/labels with trend column.
Processing label directory: /kaggle/working/exp4/labels
Updated all label files in /kaggle/working/exp4/labels with trend column.
Processing label directory: /kaggle/working/exp5/labels
Updated all label files in /kaggle/working/exp5/labels with trend column.
Processing label directory: /kaggle/working/exp6/labels
Updated all label files in /kaggle/working/exp6/labels with trend column.
Processing label directory: /kaggle/working/exp7/labels
Updated all label files in /kaggle/working/exp7/labels with trend column.
Processing label directory: /kaggle/working/exp8/labels
Updated al

In [9]:
%cd ..

/kaggle/working


In [10]:
import os
import csv
import glob

submission_rows = []

# ---------- PART 1: Image Detection Labels ----------
detection_dir = os.path.join("yolov5", "runs", "detect", "exp", "labels")
for txt_file in sorted(glob.glob(os.path.join(detection_dir, "*.txt"))):
    frame_name = os.path.basename(txt_file).replace(".txt", "")
    with open(txt_file, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 7:
                continue
            class_id, conf_det, x_min, y_min, x_max, y_max, infer_time = map(float, parts)
            class_label = "bird" if int(class_id) == 0 else "drone"
            row = [
                frame_name,
                0,
                x_min, y_min, x_max, y_max,
                class_label,
                "0",
                round(conf_det, 2),
                round(infer_time, 2),
                0,
                0,
                0,
                0,
                0
            ]
            submission_rows.append(row)

# ---------- PART 2: Video Tracking Labels ----------
tracking_base_dir = "."
exp_dirs = sorted([d for d in os.listdir(tracking_base_dir) if d.startswith("exp") and os.path.isdir(os.path.join(tracking_base_dir, d))])

for i, exp in enumerate(exp_dirs):
    labels_dir = os.path.join(tracking_base_dir, exp, "labels")
    if not os.path.isdir(labels_dir):
        continue

    # Determine device by exp index (odd = GPU, even = CPU)
    if i % 2 == 0:
        continue

    for txt_file in sorted(glob.glob(os.path.join(labels_dir, "*.txt"))):
        frame_name = os.path.basename(txt_file).replace(".txt", "")
        video_name = "_".join(frame_name.split("_")[:3])

        with open(txt_file, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 9:
                    continue
                track_id, class_id, conf_track, conf_det, x_min, y_min, x_max, y_max, direction = parts
                class_label = "bird" if int(class_id) == 0 else "drone"
                row = [
                    frame_name,
                    int(track_id),
                    float(x_min), float(y_min), float(x_max), float(y_max),
                    class_label,
                    direction,
                    round(float(conf_det), 2),
                    0,
                    round(fps_results.get(video_name, {}).get("cpu", 0), 2),
                    round(fps_results.get(video_name, {}).get("0,1", 0), 2),
                    round(float(conf_track), 2),
                    0,
                    0
                ]
                submission_rows.append(row)

# ---------- PART 3: Write Final CSV ----------
csv_path = submission_filename
header = [
    "Frame_name", "track_id", "x_min_norm", "y_min_norm", "x_max_norm", "y_max_norm",
    "class_label", "direction", "confidence_detection", "inference_time_detection (ms)",
    "FPS (CPU)", "FPS (GPU)", "confidence_track", "payload_label", "prob_harmful"
]

with open(csv_path, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    writer.writerows(submission_rows)

print(f"Submission CSV saved to {csv_path}")


Submission CSV saved to submission_detection_tracking_RGB.csv


In [26]:
!zip -r -q labels.zip /kaggle/working/exp8/labels